[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/diogoflim/MGP/blob/main/modelagem_1.ipynb)

# Modelagem e Gestão de Processos


**Prof. Diogo Ferreira de Lima Silva (TEP-UFF)**

Tema da aula - Modelagem de Processos no SimPy

# SimPy

A biblioteca SimPy apresenta um framework útil para simular o funcionamento de processos em Python. 

O usuário ganha grande liberdade para modelar processos específicos de sua organização. 

Além disso, pode integrar ao modelo as inúmeras bibliotecas existentes em python para Estatística, Ciência de Dados, Aprendeizado de Máquina, Otimização, etc.

## Importando o SimPy

Se estiver trabalhando localmente, uma vez instalado o Python, você deverá instalar a biblioteca SimPy antes de importá-la. 

- Isso pode ser feito com o comando: **pip install simpy**

No caso do Googlo Colab: **!pip install simpy**

Uma vez instalada, podemos importar a biblioteca normalmente simplesmente com: **import simpy**. 

In [2]:
#!pip install simpy
import simpy

Além do SimPy, vamos importar a biblioteca random para usar suas distribuições de probabilidades. 

In [3]:
import random

# Seu computador não gera números verdadeiramente aleatórios, mas sim, pseudoaleatórios.
# Vamos "travar" a semente de geração desses números. Assim, resultados poderão ser reproduzidos no futuro.
random.seed(10)

# Um exemplo simples: Pomodoro

Com o intuito de aumentar sua produtividade, um estudante/trabalhador resolve seguir a recomendação de uma colega sobre a metodologia Pomodoro.

- Ele fará alternadamente intervalos de trabalho e pausa (descanso, café, água, banheiro) durante o dia.

    - O tempo até a próxima parada para descanso (tempo de trabalho) segue uma distribuição uniforme que varia entre 24 e 26 minutos.
    - As pausas seguem uma exponencial de média 5 minutos.

Vamos modelar o funcionamento desse processo por 8 horas de trabalho e imprimir em nossa tela sempre que um evento (pausa ou trabalho) for iniciado.

## Criando um ambiente de simulação


O primeiro passo será criar um ambiente (environment) no SimPy. 

In [4]:
# Criando uma instância de ambiente do simpy e armazenando em env

env = simpy.Environment()

Nosso processo possui duas atividades (subprocessos): **trabalhar** e **descansar**. 

A próxima etapa é criar um **generator** (tipo especial de função) para simular o funcionamento dessas atividades. 

### Modelando o Processo

In [5]:
# Criaremos um gerador de nome pomodoro 

def pomodoro ():
    pass 

Por enquanto, não passamos nada para nosso gerador. 

Os parâmetros que ele irá receber devem incluir o ambiente criado no simpy. 

Vamos também passar como parâmetro o nome do colaborador.

In [6]:
def Pomodoro (env, nome):
    pass 

Agora, podemos trabalhar nas atividades que farão parte do nosso processo: **trabalhar** e **descansar**.

Para isso, usaremos o **yield** e os métodos **now** e **timeout** do simpy.

- O **yield** é usado em uma função geradora (em detrimento do return numa função normal). No caso do yield, guardamos um iterador (no nosso caso será o tempo de início de cada evento) que poderá ser retomado no futuro e usado em outro chamado da função geradora. 

- O método **now** do SimPy retorna o tempo de simulação no nosso ambiente enquanto o **timeout** avisa à simulação para percorrer um tempo.

In [7]:
def pomodoro (env, nome):
    
    # enquanto a simulação estiver ocorrendo
    while True:
        
        # imprima na tela o tempo de início do evento e depois percorra um tempo random.uniform(24, 26)
        print (f"{nome} inicia trabalho no tempo {env.now}")
        yield env.timeout(random.uniform(24,26))

        # imprima na tela o tempo de início do evento e depois percorra um tempo random.expovariate(1/média)
        print (f"{nome} inicia descanso no tempo {env.now}")
        yield env.timeout(random.expovariate(1/5))

Nosso gerador "pomodoro" ainda não é visto como um processo do nosso environment no SimPy. 

O próximo passo é instanciar nosso processo usando o simpy.Environment.process. 

No nosso caso, fica simplesmente **env.process**.

In [8]:
# Instância do processo
env.process(pomodoro(env, "Will_Smith"))

<Process(pomodoro) object at 0x201579357f0>

Agora, vamos rodar o nosso processo até o minuto 480, ou seja, simularemos 8h de trabalho.

In [9]:
env.run (until = 480)

Will_Smith inicia trabalho no tempo 0
Will_Smith inicia descanso no tempo 25.14280518937983
Will_Smith inicia trabalho no tempo 27.943664130161597
Will_Smith inicia descanso no tempo 53.099846732430535
Will_Smith inicia trabalho no tempo 54.253824449673225
Will_Smith inicia descanso no tempo 79.88046695238786
Will_Smith inicia trabalho no tempo 88.55515723500187
Will_Smith inicia descanso no tempo 113.86210230280423
Will_Smith inicia trabalho no tempo 114.7352358331193
Will_Smith inicia descanso no tempo 139.77657455239915
Will_Smith inicia trabalho no tempo 141.76236914065487
Will_Smith inicia descanso no tempo 166.26236249402768
Will_Smith inicia trabalho no tempo 181.53096096998934
Will_Smith inicia descanso no tempo 207.52407495506824
Will_Smith inicia trabalho no tempo 207.75197258188612
Will_Smith inicia descanso no tempo 233.4722946564587
Will_Smith inicia trabalho no tempo 238.09379086634283
Will_Smith inicia descanso no tempo 262.8570028381811
Will_Smith inicia trabalho no tem

# Modelando um Processo com Várias Atividades

Agora vamos modelar um processo um pouco mais complexo. Teremos chegadas de clientes em um serviço composto por três atividades: A, B e C.

-  Um processo de chegadas inicializa a cada chegada de um cliente. O tempo entre chegadas segue uma distribuição Exponencial com média de 5 minutos. 
    - $\frac{1}{\lambda}=5 \rightarrow \lambda = 0,2$ 

- Cada cliente segue o seguinte percurso:
    - Serviço A (tempo de processamento 3 min)
    - Serviço B (tempo de processamento de 8 min)
    - Serviço C (tempo de processamento segue uma distribuição exponencial com média 5min)

Para a realização dos serviços, precisamos de recursos. Por exemplo, atendentes, caixas, etc. 

Chamaremos os recursos de:

- colaborador_A
- colaborador_B
- colaborador_C

Para isso, utilizaremos **recursos** do SimPy, criados com **resource_name = env.Resource (env, capacity)**.

Um cliente deve esperar até que o recurso esteja disponível. Para modelar isso, usaremos o **resource_name.request()**

### Função Geradora

In [10]:
# Função geradora receberá, além do ambiente e do nome do cliente, os recursos.

def salao (env, nome, colaborador_A, colaborador_B, colaborador_C):
    
    # imprimimos na tela quando um cliente chega no estabelecimento       
    print (f"{nome} chega no estabelecimento em {env.now}")
    
    # Serviço A
    # Ao chegar, o cliente deve esperar o recurso do primeiro serviso

    with colaborador_A.request() as req_A:
        yield req_A # o cliente deve esperar um recurso do tipo colaborador_A
        print (f"{nome} inicia A em {env.now}") # imprima na tela o tempo de início
        yield env.timeout(3) # O serviço demora exatamente 3 minutos.
        print (f"{nome} finaliza A em {env.now}")

    
    # Serviço B
    with colab_B.request() as req_B:
        yield req_B
        print (f"{nome} inicia B em {env.now}")
        yield env.timeout(8) # O serviço B demora exatamente 8 minutos.
        print (f"{nome} finaliza B em {env.now}")
    
    # Serviço C
    with colab_C.request() as req_C:
        yield req_C
        print (f"{nome} inicia C em {env.now}")
        yield env.timeout(random.expovariate(1/5)) # O tempo de serviço em C segue uma exponencial de média 5.
    
    print (f"{nome} sai do estabelecimento em {env.now}")

    

### Definindo o ambiente

In [11]:
env = simpy.Environment()

### Instanciando os tipos de recurso em nosso ambiente e suas capacidades

In [12]:
colab_A = simpy.Resource(env, capacity=1)
colab_B = simpy.Resource(env, capacity=1)
colab_C = simpy.Resource(env, capacity=1)

## Processo de Chegadas


Vamos considerar que o nosso processo de chegadas é um processo de Poisson, com $\lambda = 0,2$ clientes/minuto;

Assim, o tempo entre chegadas segue uma distribuição exponencial de média $1/\lambda = 5$ minutos.

Vamos criar nosso processo de chegadas:

In [13]:
def chegadas (env):
    i=1 # número que será usado na contagem e nomeação dos clientes
    
    # Enquanto a simulação ocorrer
    while True:
        # gere um número de nossa distribuição 
        yield env.timeout(random.expovariate(1/5))

        # Um cliente entra no processo! Seu nome será "Cliente i"
        env.process(salao (env, 'Cliente %d' % i, colab_A, colab_B, colab_C))
        
        i+=1 # i = i+1


# Instanciando o processo com a função de chegadas.
env.process(chegadas(env))


<Process(chegadas) object at 0x20157942c10>

Pronto, agora basta rodar nossa simulação.

In [14]:
env.run(until=480)

Cliente 1 chega no estabelecimento em 11.423541424842186
Cliente 1 inicia A em 11.423541424842186
Cliente 2 chega no estabelecimento em 13.310582024115678
Cliente 1 finaliza A em 14.423541424842186
Cliente 1 inicia B em 14.423541424842186
Cliente 2 inicia A em 14.423541424842186
Cliente 3 chega no estabelecimento em 17.291824208921966
Cliente 2 finaliza A em 17.423541424842185
Cliente 3 inicia A em 17.423541424842185
Cliente 4 chega no estabelecimento em 20.155603800897108
Cliente 3 finaliza A em 20.423541424842185
Cliente 4 inicia A em 20.423541424842185
Cliente 5 chega no estabelecimento em 20.49161640575467
Cliente 1 finaliza B em 22.423541424842185
Cliente 1 inicia C em 22.423541424842185
Cliente 2 inicia B em 22.423541424842185
Cliente 4 finaliza A em 23.423541424842185
Cliente 5 inicia A em 23.423541424842185
Cliente 6 chega no estabelecimento em 24.88353602610119
Cliente 7 chega no estabelecimento em 25.73403230814252
Cliente 5 finaliza A em 26.423541424842185
Cliente 6 inicia A

Ótimo, já modelamos 2 processos!
 
Porém, não estamos guardando informações para calcular estatísticas que nos interessem.

---


# Monitorando o Processo

Ao analisar um processo, um gestor pode estar interessado em algumas métricas, tais como:
- Tempo de ciclo médio;
- Tempo de espera médio em fila;
- Uso dos recursos
- Trabalho em processo médio;

Para fazer tais cálculos, precisamos colher algumas informações durante a simulação.

### Vamos incrementar nosso código para guardar o tempo gasto na atividade C

In [15]:
#criando uma lista vazia
tempo_gasto_C = []

tempo_esperando_C =[]

In [16]:
def salao (env, nome, colaborador_A, colaborador_B, colaborador_C):
    #print (f"{nome} chega no estabelecimento em {env.now}")
    # Serviço A
    with colaborador_A.request() as req_A:
        yield req_A # o cliente deve esperar um recurso do tipo colaborador_A
        #print (f"{nome} inicia A em {env.now}") # imprima na tela o tempo de início
        yield env.timeout(3) # O serviço demora exatamente 3 minutos.
        #print (f"{nome} finaliza A em {env.now}")
    
    # Serviço B
    with colab_B.request() as req_B:
        yield req_B
        #print (f"{nome} inicia B em {env.now}")
        yield env.timeout(8) # O serviço B demora exatamente 8 minutos.
        #print (f"{nome} finaliza B em {env.now}")

        acaba_B = env.now

    # Serviço C
    with colab_C.request() as req_C:
        yield req_C
        #print (f"{nome} inicia C em {env.now}")
        
        p_inicio = env.now
        yield env.timeout(random.expovariate(1/5)) # O tempo de serviço em C segue uma exponencial de média 5.
        p_fim = env.now

        tempo_gasto_C.append(p_fim - p_inicio)
        tempo_esperando_C.append(p_inicio - acaba_B)
        
    #print (f"{nome} sai do estabelecimento em {env.now}")
    print (f"{nome} esperou {p_inicio - acaba_B} na fila da atividade C")
    print (f"{nome} gastou {p_fim - p_inicio} na atividade C")

In [17]:
env = simpy.Environment()
colab_A = simpy.Resource(env, capacity=1)
colab_B = simpy.Resource(env, capacity=1)
colab_C = simpy.Resource(env, capacity=1)
env.process(chegadas(env))
env.run(until = 60000)

Cliente 1 esperou 0.0 na fila da atividade C
Cliente 1 gastou 0.08226210443761772 na atividade C
Cliente 2 esperou 0.0 na fila da atividade C
Cliente 2 gastou 1.943756968837853 na atividade C
Cliente 3 esperou 0.0 na fila da atividade C
Cliente 3 gastou 4.711971012669444 na atividade C
Cliente 4 esperou 0.0 na fila da atividade C
Cliente 4 gastou 13.723758764044732 na atividade C
Cliente 5 esperou 5.723758764044732 na fila da atividade C
Cliente 5 gastou 1.0642784246346366 na atividade C
Cliente 6 esperou 0.0 na fila da atividade C
Cliente 6 gastou 4.241927529946544 na atividade C
Cliente 7 esperou 0.0 na fila da atividade C
Cliente 7 gastou 2.8929594617060417 na atividade C
Cliente 8 esperou 0.0 na fila da atividade C
Cliente 8 gastou 10.185432999406473 na atividade C
Cliente 9 esperou 2.1854329994064727 na fila da atividade C
Cliente 9 gastou 0.08089139705820969 na atividade C
Cliente 10 esperou 0.0 na fila da atividade C
Cliente 10 gastou 4.26940996401423 na atividade C
Cliente 11 e

In [18]:
import numpy as np

print(f"Em média, um cliente o serviço C dura {np.mean(tempo_gasto_C)} minutos")
print(f"Em média, um cliente espera {np.mean(tempo_esperando_C)} minutos pelo serviço C")


Em média, um cliente o serviço C dura 5.019555734043165 minutos
Em média, um cliente espera 2.8508108161566366 minutos pelo serviço C


# Exercício Prático 2

### Modelo M/M/1

Vamos modelar um sistema faz parte do curso de Pesquisa Operacional III, o sistema de filas M/M/1.

- Taxa de chegadas $\lambda = 0,5$
- Taxa de atendimento $\mu = 1$

Guarde o valor do tempo de espera em fila e do tempo no sistema (tempo de ciclo).

Simule por 60000 minutos. 

Os resultados batem com o esperado? Faça os cálculos com as fórmulas clássicas de Teoria das Filas?

In [129]:
def fila_MM1 (env, nome, atendente):
    pass
